# ID3 Classification Trees: Perfect Split with Information Gain - Lab

## Introduction

In this lab, we will simulate the example from the previous lesson in Python. You will write functions to calculate entropy and IG which will be used for calculating these uncertainty measures and deciding upon creating a split using information gain while growing an ID3 classification tree. You will also write a general function that can be used for other (larger) problems as well. So let's get on with it.

## Objectives

In this lab you will: 

- Write functions for calculating entropy and information gain measures  
- Use entropy and information gain to identify the attribute that results in the best split at each node


## Problem

You will use the same problem about deciding whether to go and play tennis on a given day, given the weather conditions. Here is the data from the previous lesson:

|  outlook | temp | humidity | windy | play |
|:--------:|:----:|:--------:|:-----:|:----:|
| overcast | cool |   high   |   Y   |  yes |
| overcast | mild |  normal  |   N   |  yes |
|   sunny  | cool |  normal  |   N   |  yes |
| overcast |  hot |   high   |   Y   |  no  |
|   sunny  |  hot |  normal  |   Y   |  yes |
|   rain   | mild |   high   |   N   |  no  |
|   rain   | cool |  normal  |   N   |  no  |
|   sunny  | mild |   high   |   N   |  yes |
|   sunny  | cool |  normal  |   Y   |  yes |
|   sunny  | mild |  normal  |   Y   |  yes |
| overcast | cool |   high   |   N   |  yes |
|   rain   | cool |   high   |   Y   |  no  |
|   sunny  |  hot |  normal  |   Y   |  no  |
|   sunny  | mild |   high   |   N   |  yes |


## Write a function `entropy(pi)` to calculate total entropy in a given discrete probability distribution `pi`

- The function should take in a probability distribution `pi` as a list of class distributions. This should be a list of two integers, representing how many items are in each class. For example: `[4, 4]` indicates that there are four items in each class, `[10, 0]` indicates that there are 10 items in one class and 0 in the other. 
- Calculate and return entropy according to the formula: $$Entropy(p) = -\sum (P_i . log_2(P_i))$$

In [14]:
from math import log 
import numpy as np

def entropy(pi):
    """
    return the Entropy of a probability distribution:
    entropy(p) = - SUM (Pi * log(Pi) )
    """

    ent = 0
    for p in pi:
        p = p / sum(pi)
        if p != 0:
            ent += -p*log(p, 2)
    
    return ent


# Test the function 

print(entropy([1, 1])) # Maximum Entropy e.g. a coin toss
print(entropy([0, 6])) # No entropy, ignore the -ve with zero , it's there due to log function
print(entropy([2, 10])) # A random mix of classes

# 1.0
# -0.0
# 0.6500224216483541

1.0
0.0
0.6500224216483541


## Write a function `IG(D,a)` to calculate the information gain 

- As input, the function should take in `D` as a class distribution array for target class, and `a` the class distribution of the attribute to be tested
- Using the `entropy()` function from above, calculate the information gain as:

$$gain(D,A) = Entropy(D) - \sum(\frac{|D_i|}{|D|}.Entropy(D_i))$$

where $D_{i}$ represents distribution of each class in `a`.


In [20]:
def IG(D, a):
    """
    return the information gain:
    gain(D, A) = entropy(D)− SUM( |Di| / |D| * entropy(Di) )
    """
    
    total = 0
    for Di in a:
        total += (abs(sum(Di)) / sum(D)) * entropy(Di)
    
    gain = entropy(D) - total
    return gain
    


# Test the function
# Set of example of the dataset - distribution of classes
test_dist = [6, 6] # Yes, No
# Attribute, number of members (feature)
test_attr = [ [4,0], [2,4], [0,2] ] # class1, class2, class3 of attr1 according to YES/NO classes in test_dist

print(IG(test_dist, test_attr))

# 0.5408520829727552

0.5408520829727552


## First iteration - Decide the best split for the root node

- Create the class distribution `play` as a list showing frequencies of both classes from the dataset
- Similarly, create variables for four categorical feature attributes showing the class distribution for each class with respect to the target classes (yes and no)
- Pass the play distribution with each attribute to calculate the information gain

In [26]:
# Your code here
import pandas as pd
import io

data = '''outlook	temp	humidity	windy	play
overcast	cool	high	Y	yes
overcast	mild	normal	N	yes
sunny	cool	normal	N	yes
overcast	hot	high	Y	no
sunny	hot	normal	Y	yes
rain	mild	high	N	no
rain	cool	normal	N	no
sunny	mild	high	N	yes
sunny	cool	normal	Y	yes
sunny	mild	normal	Y	yes
overcast	cool	high	N	yes
rain	cool	high	Y	no
sunny	hot	normal	Y	no
sunny	mild	high	N	yes'''

buffer = io.StringIO(data)

df = pd.read_csv(buffer, delimiter='\t')

df

,outlook,temp,humidity,windy,play
0,overcast,cool,high,Y,yes
1,overcast,mild,normal,N,yes
2,sunny,cool,normal,N,yes
3,overcast,hot,high,Y,no
4,sunny,hot,normal,Y,yes
5,rain,mild,high,N,no
6,rain,cool,normal,N,no
7,sunny,mild,high,N,yes
8,sunny,cool,normal,Y,yes
9,sunny,mild,normal,Y,yes


In [45]:
df[(df['outlook'] == the_outlook) & (df['play'] == 'yes')]


,outlook,temp,humidity,windy,play
0,overcast,cool,high,Y,yes
1,overcast,mild,normal,N,yes
10,overcast,cool,high,N,yes


In [52]:
play = [ len(df[df['play'] == 'yes']), len(df[df['play'] == 'no']) ]

outlook = []
for feature in list(df.outlook.unique()):
    outlook.append([len(df[(df['outlook'] == feature) & (df['play'] == 'yes')]),
                   len(df[(df['outlook'] == feature) & (df['play'] == 'no')])])


temperature = []
for feature in list(df.temp.unique()):
    temperature.append([len(df[(df['temp'] == feature) & (df['play'] == 'yes')]),
                   len(df[(df['temp'] == feature) & (df['play'] == 'no')])])
    
humidity = []
for feature in list(df.humidity.unique()):
    humidity.append([len(df[(df['humidity'] == feature) & (df['play'] == 'yes')]),
                   len(df[(df['humidity'] == feature) & (df['play'] == 'no')])])

wind = []
for feature in list(df.windy.unique()):
    wind.append([len(df[(df['windy'] == feature) & (df['play'] == 'yes')]),
                   len(df[(df['windy'] == feature) & (df['play'] == 'no')])])
# Information Gain:

print ('Information Gain:\n' )
print('Outlook:', IG(play, outlook))
print('Temperature:', IG(play, temperature))
print('Humidity:', IG(play, humidity))
print('Wind:,', IG(play, wind))

# Outlook: 0.41265581953400066
# Temperature: 0.09212146003297261
# Humidity: 0.0161116063701896
# Wind:, 0.0161116063701896

Information Gain:

Outlook: 0.41265581953400066
Temperature: 0.0921214600329725
Humidity: 0.0161116063701896
Wind:, 0.0161116063701896


We see here that the outlook attribute gives us the highest value for information gain, hence we choose this for creating a split at the root node. So far, we've built the following decision tree:
<img src='images/outlook.png'  width ="650"  >


## Second iteration

Since the first iteration determines what split we should make for the root node of our tree, it's pretty simple. Now, we move down to the second level and start finding the optimal split for each of the nodes on this level. The first branch (edge) of three above that leads to the "Sunny" outcome. Of the temperature, humidity and wind attributes, find which one provides the highest information gain.

Follow the same steps as above. Remember, we have 6 positive examples and 1 negative example in the "sunny" branch.

In [54]:
# Your code here 
sunny = [6, 1]

df_sunny = df[df['outlook'] == 'sunny']

temperature = []
for feature in list(df_sunny.temp.unique()):
    temperature.append([len(df_sunny[(df_sunny['temp'] == feature) & (df_sunny['play'] == 'yes')]),
                   len(df_sunny[(df_sunny['temp'] == feature) & (df_sunny['play'] == 'no')])])
    
humidity = []
for feature in list(df_sunny.humidity.unique()):
    humidity.append([len(df_sunny[(df_sunny['humidity'] == feature) & (df_sunny['play'] == 'yes')]),
                   len(df_sunny[(df_sunny['humidity'] == feature) & (df_sunny['play'] == 'no')])])

wind = []
for feature in list(df_sunny.windy.unique()):
    wind.append([len(df_sunny[(df_sunny['windy'] == feature) & (df_sunny['play'] == 'yes')]),
                   len(df_sunny[(df_sunny['windy'] == feature) & (df_sunny['play'] == 'no')])])



# Information Gain:
print ('Information Gain:\n' )

print('Temperature:', IG(sunny, temperature))
print('Humidity:', IG(sunny, humidity))
print('Wind:,', IG(sunny, wind))

# Temperature: 0.3059584928680418
# Humidity: 0.0760098536627829
# Wind: 0.12808527889139443

Information Gain:

Temperature: 0.3059584928680418
Humidity: 0.0760098536627829
Wind:, 0.12808527889139443


We see that temperature gives us the highest information gain, so we'll use it to split our tree as shown below:

<img src='images/temp.png'  width ="650"  >


Let's continue. 

## Third iteration

We'll now calculate splits for the 'temperature' node we just created for days where the weather is sunny. Temperature has three possible values: [Hot, Mild, Cool]. This means that for each of the possible temperatures, we'll need to calculate if splitting on windy or humidity gives us the greatest possible information gain.

Why are we doing this next instead of the rest of the splits on level 2? Because a decision tree is a greedy algorithm, meaning that the next choice is always the one that will give it the greatest information gain. In this case, evaluating the temperature on sunny days gives us the most information gain, so that's where we'll go next.

## All other iterations

What happens once we get down to a 'pure' split? Obviously, we stop splitting. Once that happens, we go back to the highest remaining uncalculated node and calculate the best possible split for that one. We then continue on with that branch, until we have exhausted all possible splits or we run into a split that gives us 'pure' leaves where all 'play=Yes' is on one side of the split, and all 'play=No' is on the other.

## Summary 

This lab should have helped you familiarize yourself with how decision trees work 'under the hood', and demystified how the algorithm actually 'learns' from data by: 

- Calculating entropy and information gain
- Figuring out the next split you should calculate ('greedy' approach) 